In [1]:
# Recreated from Google Admin Toolbox https://toolbox.googleapps.com/apps/encode_decode/  *minus SAML features
# Added ability to skip encoding and convert input to hex or binary directly
# Intended for render via JupyterLab's built-in Viola
# Created by M-A-P-E https://github.com/M-A-P-E

In [2]:
import ipywidgets as widgets
import base64
import urllib
import hashlib
import json
import quopri
import codecs

In [3]:
# Base64 Encode / Decode

def base64_encode(text):
    return str(base64.b64encode(text.encode('ascii')), 'utf-8')

def base64_decode(text):
    return str(base64.b64decode(text), 'utf-8')

In [4]:
# Base64Url Encode / Decode

def base64url_encode(text):
    text = text.encode('utf-8')
    return base64.urlsafe_b64encode(text).rstrip(b'=')

def base64url_decode(text):
    padding = b'=' * (4 - (len(text) % 4))
    return base64.urlsafe_b64decode(text.encode('utf-8') + padding)


In [5]:
# URL Encode / Decode

def url_encode(text):
    return urllib.parse.quote(text)

def url_decode(text):
    return urllib.parse.unquote(text)

In [6]:
# MD5 Hash

def md5(text):
    return hashlib.md5(text.encode('utf-8')).hexdigest()

In [7]:
# Pretty JSON

def pretty_json(text):
    parsed = json.loads(text)
    return json.dumps(parsed, indent=4, sort_keys=True)

In [8]:
# Quoted-Printable Encode / Decode

def quoted_printable_encode(text):
    return quopri.encodestring(text.encode('utf-8'))

def quoted_printable_decode(text):
    return quopri.decodestring(text)

In [9]:
# UTF-16 Decode

def utf16_decode(text):
    return codecs.decode(text, 'unicode-escape')

In [10]:
# Input and Output Textarea widgets
input_textarea = widgets.Textarea(placeholder='Paste the text to encode/decode below', layout=widgets.Layout(width='auto', height='250px'))
output_textarea = widgets.Textarea(layout=widgets.Layout(width='auto', height='250px'))

# Encode / Decode RadioButtons widget
options_radiobuttons = widgets.RadioButtons(options=['Convert Output', 'Base64 Encode', 'Base64 Decode', 'Base64Url Encode', 'Base64Url Decode', 'URL Encode', 'URL Decode', 'MD5 Hash', 'Pretty JSON', 'Quoted-Printable Encode', 'Quoted-Printable Decode', 'UTF16 Decode'])

# Output Options Dropdown widget
output_options_dropdown = widgets.Dropdown(options=['Plain Text', 'Hexadecimal Output', 'Binary Output'])

# Submit Button widget
submit_button = widgets.Button(description='SUBMIT', disabled=False, button_style='success')



# Display widgets in a defined structure using Python wrapped CSS grid
grid = widgets.GridspecLayout(3, 4)
grid[0, 0:3] = input_textarea
grid[1, 0:3] = output_textarea
grid[0, 3] = options_radiobuttons
grid[1, 3] = output_options_dropdown
grid[2, 3] = submit_button

grid

GridspecLayout(children=(Textarea(value='', layout=Layout(grid_area='widget001', height='250px', width='auto')…

In [13]:
# Submit Button onclick event

def format_output(text):
    if type(text) == bytes:
        return str(text, 'utf-8')
    elif type(text) == str:
        return text

def output_style(text):
    if output_options_dropdown.value == 'Plain Text':
        return text
    if output_options_dropdown.value == 'Hexadecimal Output':
        _ = bytes(text, 'utf-8')
        return _.hex()
    if output_options_dropdown.value == 'Binary Output':
        return (' '.join(["{0:b}".format(x) for x in bytes(text, 'utf-8')]))

def submit_button_onclick(_):
    text = input_textarea.value
    
    if text == '':
        output_textarea.value = 'Error: Please do not leave this blank.'
    
    try:
        if options_radiobuttons.value == 'Convert Output':
            output_textarea.value = output_style(text)
        if options_radiobuttons.value == 'Base64 Encode':
            output_textarea.value = output_style(format_output(base64_encode(text)))
        if options_radiobuttons.value == 'Base64 Decode':
            output_textarea.value = output_style(format_output(base64_decode(text)))
        if options_radiobuttons.value == 'Base64Url Encode':
            output_textarea.value = output_style(format_output(base64url_encode(text)))
        if options_radiobuttons.value == 'Base64Url Decode':
            output_textarea.value = output_style(format_output(base64url_decode(text)))
        if options_radiobuttons.value == 'URL Encode':
            output_textarea.value = output_style(format_output(url_encode(text)))
        if options_radiobuttons.value == 'URL Decode':
            output_textarea.value = output_style(format_output(url_decode(text)))
        if options_radiobuttons.value == 'MD5 Hash':
            output_textarea.value = output_style(format_output(md5(text)))
        if options_radiobuttons.value == 'Quoted-Printable Encode':
            output_textarea.value = output_style(format_output(quoted_printable_encode(text)))
        if options_radiobuttons.value == 'Quoted-Printable Decode':
            output_textarea.value = output_style(format_output(quoted_printable_decode(text)))
    except Exception as error:
        output_textarea.value = error
        
    if options_radiobuttons.value == 'Pretty JSON':
        try:
            output_textarea.value = output_style(format_output(pretty_json(text)))
        except Exception:
            output_textarea.value = 'Error: Not a valid JSON string.'
        
    if options_radiobuttons.value == 'UTF16 Decode':
        try:
            output_textarea.value = output_style(format_output(utf16_decode(text)))
        except Exception:
            output_textarea.value = 'Error: Unable to convert the UTF-16 escaped string. Make sure the format is like "\u30ab\u30ec....."'

submit_button.on_click(submit_button_onclick)